In [1]:
from nltk import word_tokenize, sent_tokenize, tokenize
import numpy as np
import nltk
import pandas as pd
from nltk.util import ngrams
stop = nltk.corpus.stopwords.words('portuguese')
from IPython.display import display, HTML

nltk.download('punkt')
from collections import Counter

# Contagens dos unigramas



[nltk_data] Downloading package punkt to
[nltk_data]     /home/brunorosilva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
corpus = open('DomCasmurro.txt','r').read()

In [3]:
[1, 3] + ["2"]

[1, 3, '2']

In [4]:
a = {}
a["B"] = []
a["B"].append(2)
a["C"] = []
a["C"].append(3)
a["B"].append(2)
a["B"].append(2)

In [5]:
a

{'B': [2, 2, 2], 'C': [3]}

In [6]:
for k in a:
    print(a[k])

[2, 2, 2]
[3]


In [7]:
a = [1, 2,3]
a[:-1]

[1, 2]

In [167]:
class model():
    def __init__(self, corpus):
        self.corpus = corpus
        self.preprocess_done = False
        self.stop = nltk.corpus.stopwords.words('portuguese')
        self.unigram=[]
        self.bigram=[]
        self.trigram=[]
        self.lst_sents = []
        

    def _tokenizar(self, s):
        return tokenize.word_tokenize(s, language='portuguese')

    def _limpar(self, lista):
        return [i.lower() for i in lista if i.isalpha()]
    
    def _achatar(self, lista):
        return [i for sublista in lista for i in sublista]
    
    def _remover_pontuacao(self):
        self.pontos = ['...', ':', ';', '!', '?']
        for i in self.pontos:
            self.corpus = self.corpus.replace(i, '.')
        
        
    def _remover_barra_n(self):
        self.corpus_nl_removed = ""
        for line in self.corpus:
            line_nl_removed = line.replace("\n", " ")      #removes newlines
            self.corpus_nl_removed += line_nl_removed
            
    def _remover_pontuacao_e_barra_n(self):
        self.corpus_limpo = "".join([char for char in self.corpus_nl_removed if char not in (self.pontos + ['\n'])])
        
    def _estatisticas_corpus(self):
        self.sents = sent_tokenize(self.corpus_limpo)
        self.words = word_tokenize(self.corpus_limpo)
        stats_df = pd.DataFrame({
            "Sentências":[len(self.sents)],
            "Palavras":[len(self.words)],
            "Média de palavras por sentença":[round(len(self.words)/len(self.sents))],
            "Quantidade de palavras únicas":[len(set(self.words))],
        }, index=["Estatísticas"]).transpose()
        
        display(stats_df)
        
        
    def _prever(self, palavra):
        lista = [k for k in c_bigr.items() if k[0][0] == palavra]
        ordem = sorted(lista, key=lambda x: x[1], reverse=True)
        previsao = [p[0][1] for p in ordem[:3]]
        return previsao

    def _criar_lst_sents(self):        
        for sentence in self.sents:
            if len(sentence) > 0:
                self.lst_sents.append(self._limpar(self._tokenizar(sentence)))
        for i in range(len(self.lst_sents)):
            self.lst_sents[i] = ["<s>"] + self.lst_sents[i] + ["</s>"]
    
    def _remover_primeiros(self, n=10):
        self.lst_sents = self.lst_sents[10:]
    
    def preprocess(self):
        
        if self.preprocess_done == True:
            print("O preprocessamento já foi feito, você pode criar os modelos diretamente")
        
        else:
            print("Começando o preprocessamento")
            
            self._remover_pontuacao()
            print("Passo 1 Check")
            self._remover_barra_n()
            print("Passo 2 Check")
            self._remover_pontuacao_e_barra_n()
            print("Passo 3 Check")
            self._estatisticas_corpus()
            print("Passo 4 Check")
            self._criar_lst_sents()
            print("Passo 5 Check")
            self._remover_primeiros()
            print("Passo 6 Check")
            
                
    def criar_ngrams(self, n_list=[1, 2, 3]):
        self.ngrams = {}
        
        for n in n_list:
            self.ngrams[str(n)+"gram"] = []
            
            for s in self.lst_sents:
                if s == "." and n==1:
                    pass
                elif n==1:
                    self.ngrams[str(n)+"gram"].append(s) 
                else:
                    self.ngrams[str(n)+"gram"].append(list(ngrams(s, n)))
            
    def calcular_freqs(self):
        self.ngrams_counters = {}
        for ngram in self.ngrams:
            c_ngram = Counter(self._achatar(self.ngrams[ngram]))
            d = pd.DataFrame.from_dict(c_ngram, orient='index', columns=[ngram])
            display(d.sort_values(ngram, ascending=False))
            self.ngrams_counters[ngram] = c_ngram

    def _p_onegram(self, p):        
        
        return self.ngrams_counters["1gram"][p] / sum(self.ngrams_counters["1gram"].values())
        
        
    def _p_bigram(self, p1, p2):        
        return self.ngrams_counters["2gram"][(p1, p2)] / self.ngrams_counters["1gram"][p1]
        
    def _p_trig(self, p1, p2, p3):        
        return self.ngrams_counters["3gram"][(p1, p2, p3)] / self.ngrams_counters["2gram"][(p1, p2)]
    
    
    def calcular_probs(self, p_list):
        self.ngrams_prob = {}
        
        ### solução besta só pra 1 <= n <= 3
        # futuramente vou criar uma solução recursiva para o problema
        
        if len(p_list) == 1:
            prob = self._p_onegram(p_list[0])
            self.ngrams_prob["1gram"] = prob
        elif len(p_list) == 2:
            prob = self._p_bigram(p_list[0], p_list[1])
            self.ngrams_prob["2gram"] = prob
        else:
            prob = self._p_trig(p_list[0], p_list[1], p_list[2])
            self.ngrams_prob["3gram"] = prob
                    
                
    def smooth(self):
        self.ngrams_voc = {}
        
        self.total_ngrams = {}
        self.total_voc = {}
        
        self.ngrams_prob = {}
        for ngram in self.ngrams:
            self.ngrams_voc[ngram] = set([])
            
            for gram in self.ngrams[ngram]:
                if self._achatar(gram) not in self._achatar(self.ngrams_voc[ngram]):
                    self.ngrams_voc[ngram] = self._achatar(gram)        
        
            self.total_ngrams[ngram] = -1
            self.total_voc[ngram] = -1
        

            self.ngrams_prob[ngram] = []
            
            for ngram_v in self.ngrams_voc[ngram]:
                tlist = [ngram_v]
                tlist.append(self.ngrams[ngram].count(ngram_v))
                self.ngrams_prob[ngram].append(tlist)
        
        
            self.total_ngrams[ngram] = len(self.ngrams[ngram])
            self.total_voc[ngram] = len(self.ngrams_voc[ngram])                       

            for ngram_p in self.ngrams_prob[ngram]:
                self.ngrams_probngram_p[-1] = (ngram_p[-1]+1)/(self.total_ngrams[ngram]+self.total_voc[ngram])             #add-1 smoothing

    
            self.ngrams_prob[ngram] = sorted(self.ngrams_prob[ngram], key = lambda x:x[1], reverse = True)
    
    def calculate_preds(self, word, limit=40):
        for ngram in self.ngrams_counters:
            lista = [k for k in self.ngrams_counters[ngram].items() if k[0][0] == word]
            ordem = sorted(lista, key=lambda x: x[1], reverse=True)
            
            i = 1
            previsao = []
            for p in ordem[:limit]:
                previsao.append(p[0][1])
                
                if p[0][1] == "</s>":
                    print("quebrou por conta do final")
                    break
                #if i == limit:
                #    print("quebrou por conta do limite")
                #    break
                #i += 1
            print(previsao)

In [168]:
m = model(corpus)

In [169]:
m.preprocess()

Começando o preprocessamento
Passo 1 Check
Passo 2 Check
Passo 3 Check


,Estatísticas
Sentências,5760
Palavras,83048
Média de palavras por sentença,14
Quantidade de palavras únicas,11196


Passo 4 Check
Passo 5 Check
Passo 6 Check


In [170]:
m.criar_ngrams()

In [171]:
m.calcular_freqs()

,1gram
<s>,5750
</s>,5750
que,2663
a,2579
e,2191
...,...
rezando,1
miquelina,1
joanna,1
nhõ,1


,2gram
"(<s>, não)",368
"(<s>, a)",279
"(<s>, mas)",226
"(<s>, o)",196
"(que, não)",190
...,...
"(<s>, trabalhei)",1
"(proprios, </s>)",1
"(sentidos, proprios)",1
"(e, sentidos)",1


,3gram
"(<s>, josé, dias)",49
"(de, minha, mãe)",44
"(<s>, minha, mãe)",36
"(<s>, o, que)",30
"(josé, dias, </s>)",27
...,...
"(sensação, exquisita, </s>)",1
"(<s>, um, fluido)",1
"(um, fluido, </s>)",1
"(<s>, aquella, ameaça)",1


In [172]:
m.calcular_probs(["<s>", "um", "fluido"])

In [173]:
m.ngrams_prob

{'3gram': 0.01639344262295082}

In [175]:
m.calculate_preds("<s>")

[]
quebrou por conta do final
['não', 'a', 'mas', 'o', 'e', 'era', 'eu', 'capitú', 'que', 'é', 'quando', 'se', 'os', 'um', 'ao', 'as', 'você', 'tambem', 'pois', 'em', 'josé', 'já', 'como', 'foi', 'no', 'tudo', '</s>']
['josé', 'minha', 'o', 'não', 'não', 'mas', 'prima', 'um', 'mas', 'mas', 'oh', 'é', 'tio', 'mas', 'não', 'ao', 'creio', 'era', 'era', 'os', 'que', 'não', 'como', 'era', 'se', 'não', 'você', 'não', 'a', 'tambem', 'já', 'é', 'não', 'era', 'ha', 'uma', 'eu', 'if', 'mas', 'o']


# preprocessing
1. Trocar as pontuações de final de frase por "."
2. Remover 